In [1]:
# Librerías usadas:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
# Levanto los rangos consumo:
dfrc = pd.read_csv('./Rango_cons.csv', encoding = 'latin1',  decimal=',')

In [3]:
# Levanto el stock existente en una carpeta:

ruta = './Stock_SanCor/'
contenido = os.listdir(ruta)

print("Inicio del proceso: " + datetime.now().strftime('%H:%M:%S'))

# Dataframe con el stock:
dfs = pd.DataFrame()

archivos = []
for fichero in contenido:
    if os.path.isfile(os.path.join(ruta, fichero)) and fichero.endswith('.xlsx'):
        df_temp = pd.read_excel(ruta+fichero)
        dfs = pd.concat([dfs, df_temp], axis=0)
        print("Se cargó el archivo: ", fichero, "a las: " + datetime.now().strftime('%H:%M:%S'))

df_temp = []
print("Fin de la carga de archivos")

Inicio del proceso: 12:31:34
Se cargó el archivo:  20.1 Stock 202010.xlsx a las: 12:34:22
Se cargó el archivo:  20.1 Stock 202011.xlsx a las: 12:37:48
Se cargó el archivo:  20.1 Stock 202012.xlsx a las: 12:40:42
Se cargó el archivo:  20.1 Stock 202101.xlsx a las: 12:43:33
Se cargó el archivo:  20.1 Stock 202102.xlsx a las: 12:46:24
Se cargó el archivo:  20.1 Stock 202103.xlsx a las: 12:49:17
Se cargó el archivo:  20.1 Stock 202104.xlsx a las: 12:52:16
Se cargó el archivo:  20.1 Stock 202105.xlsx a las: 12:55:16
Se cargó el archivo:  20.1 Stock 202106.xlsx a las: 12:58:21
Se cargó el archivo:  20.1 Stock 202107.xlsx a las: 13:01:40
Se cargó el archivo:  20.1 Stock 202108.xlsx a las: 13:05:00
Se cargó el archivo:  20.1 Stock 202109.xlsx a las: 13:08:24
Fin de la carga de archivos


In [4]:
# Cambiar nombre a las columnas
dfs.rename( columns = {    'Periodo ID': 'Periodo',
                           'Persona ID': 'Persona',
                           'Persona Fecha Nacimiento': 'Fecha_nac',
                           'Discapacitado ID': 'Discapacidad',
                           'Marca Persona con Diabetes DESC': 'Diabetes',
                           'Marca Persona con Hepatitis DESC': 'Hepatitis',
                           'Marca Persona con PMI DESC': 'PMI',
                           'Marca Persona con Tratamiento Cronico DESC': 'Cronico',
                           'Marca Persona con Tratamiento Oncológico DESC':  'Oncologico',
                           'Sexo ID': 'Sexo',
                           'Plan Sin Segmento ID': 'Plan',
                           'Posicion Asociado DESC': 'Posicion Asociado',
                           'Zona de Promocion DESC': 'Zona de Promocion',
                           'Segmento Agrupado DESC': 'Segmento',
                           'Plan Agrupado Sin Segmento DESC': 'Plan Agrupado',
                           'Stock Asociados Salud': 'Stock',
                           'Edad Periodo ID': 'Edad',
                           'ACE Stock Asociados Salud': 'ACE',
                           'Consumos Costo con Subsidios SSS': 'Consumo3',
                           'Recaudacion Costo con Otros Subsidios': 'Recaudacion3'                      
                           }, inplace=True)

In [5]:
dfrc.columns

Index(['Persona', 'Stock', 'Consumo3', 'GC', 'Cons_mensualizado',
       'Rango_Cons'],
      dtype='object')

In [6]:
# Pensar la decisión a tomar con "sin valor"
dfrc.Rango_Cons.unique()

array(['Alto', 'Medio', 'Muy Alto', 'Bajo', 'Muy Bajo'], dtype=object)

In [7]:
# Pegamos a dfs el rango de consumo de los encontrados:
dfs = pd.merge(left = dfs, right = dfrc[['Persona', 'GC', 'Rango_Cons']], left_on='Persona', right_on='Persona', how = "left")

In [8]:
# Existen casos con nan a reemplazar (decisión leo)
# Al ser personas encontradas en el stock y no encontradas en el consumo ponemos: No GC y Rango Cons == Muy Bajo
values = {"GC": "NO GC", "Rango_Cons": "Muy Bajo"}
dfs = dfs.fillna(value=values)

In [9]:
# Realizamos las transformaciones:

# Reemplazar GC = S por 1 y N por 0
dfs['GC'] = dfs['GC'].replace(['NO GC', 'GC'], [0, 1])


# Definiciones:
# Proporción mínima de meses con diagnóstico para ser considerado con algún diagnóstico:
prop_limite = 0.4
# Se puede agregar los campos considerados como diag..

# Reemplazo valores de filas con replace:
dfs['Diabetes'] = dfs['Diabetes'].replace(['No Diabético', 'Diabético'], [0, 1])
dfs['Hepatitis'] = dfs['Hepatitis'].replace(['Sin Hepatitis', 'Con Hepatitis'], [0, 1])
dfs['PMI'] = dfs['PMI'].replace(['NO', 'SI'], [0, 1])
dfs['Cronico'] = dfs['Cronico'].replace(['NO', 'SI'], [0, 1])
dfs['Oncologico'] = dfs['Oncologico'].replace(['Sin Marca', 'Paciente Oncológico'], [0, 1])
dfs['GC'] = dfs['GC'].replace(['NO GC', 'GC'], [0, 1])
dfs['Discapacidad'] = dfs['Discapacidad'].replace(['N', 'S'], [0, 1])

# Creo una columna de resultado que marque 1 si se tiene cualquier diagnostico
# Se tendría que mejorar para generalizar:
suma = (dfs['Diabetes'] + dfs['Hepatitis'] + dfs['PMI'] + 
        dfs['Cronico'] + dfs['Oncologico'] + dfs['GC'] + dfs['Discapacidad'])
dfs['algun_diag'] = np.where( suma > 0, 1, 0)

# Creo una auxiliar que resuma por persona el stock y cantidad de veces con diag.
# Agrupo por persona las diferentes métricas en otro df para luego cruzar con la base original
filas = ['Persona','Rango_Cons']
columnas = ['Stock','Diabetes', 'Hepatitis', 'PMI','Cronico', 'Oncologico', 'GC', 'Discapacidad', 'algun_diag']
aux_diag = pd.DataFrame(dfs.pivot_table(columnas , filas, aggfunc = np.sum))


# Genero una columna que calcule el % de veces que se tuvo algún diagnóstico:
aux_diag['prop_diag'] = aux_diag['algun_diag'] / aux_diag['Stock']
aux_diag.tail()

# Creamos la marca a cada persona
# Analizando las diferentes posibilidades llegué a la conclusión que si la proporción es mayor a 40% se debe poner "Con Diagnóstico"
aux_diag['Marca_Diagnostico'] = np.where(aux_diag['prop_diag'] > prop_limite, 'Con Diagnostico','Sin Diagnostico')

# Reseteo los multi index:
aux_diag = aux_diag.reset_index()

In [10]:
aux_diag.head()

,Persona,Rango_Cons,Cronico,Diabetes,Discapacidad,GC,Hepatitis,Oncologico,PMI,Stock,algun_diag,prop_diag,Marca_Diagnostico
0,215,Alto,0,0,0,0,0,0,0,9,0,0.0,Sin Diagnostico
1,218,Medio,0,0,0,0,0,0,0,12,0,0.0,Sin Diagnostico
2,225,Muy Alto,0,3,0,0,0,0,0,3,3,1.0,Con Diagnostico
3,246,Medio,0,0,0,0,0,0,0,12,0,0.0,Sin Diagnostico
4,248,Medio,0,0,0,0,0,0,0,12,0,0.0,Sin Diagnostico


In [11]:
aux_diag.to_csv("Diagnosticos.csv", encoding = 'latin1',  decimal=',')